In [ ]:
from langgraph.graph import StateGraph, MessagesState, START, END
from IPython.display import Image,Markdown,display_markdown ,display
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage,SystemMessage, ToolMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool
from tavily import TavilyClient
from typing import Literal, Union, List

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your TAVILY API key: ")
    

# TOOLS

In [ ]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

@tool
def internet_search(
    query: str,
    max_results: int = 5,
    start_date: str = None,
    end_date: str = None,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    """Run a tavily web search
    Args:
        query: The search query. str
        max_results: The maximum number of results to return. int
        start_date: The start date for the search in YYYY-MM-DD format. str
        end_date: The end date for the search in YYYY-MM-DD format. str
        topic: The topic of the search. Can be "general", "news", "finance". str
        include_raw_content: Whether to include the raw content of the results. boolean
    """
    return tavily_client.search(
        query,
        max_results=max_results,
        start_date= start_date,
        end_date= end_date,
        include_raw_content=include_raw_content,
        topic=topic,
    )

In [ ]:
@tool
def extract_text_from_url(
    urls: Union[List[str], str],
    extract_depth: Literal["basic", "advanced"] = "advanced",
    format: Literal["markdown", "text"] = "markdown",
):
    """Extract text from a given URL using Tavily.
    Args:
        urls: The URL/urls to extract text from. list[str] or str
        extract_depth: The depth of extraction. Can be "basic" or "advanced". str
        format: type of format to extract. Can be "markdown" or "text". str
    """
    return tavily_client.extract(
        urls,
        extract_depth=extract_depth,
        format=format,
    )

# LLM

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    
)
tools = [internet_search, extract_text_from_url]
tools_by_name = {tool.name: tool for tool in tools}
llmwithtools = llm.bind_tools(tools)

# NODE

In [ ]:
system_msg = SystemMessage(content=
    "You are a helpful Agentic AI.Currently You have access to these tools "
    "[internet_search, extract_text_from_url]. Use them as per need. "
    "also you know which tool to use when.\n\n"
    
    "Citation Rules (Apply ONLY when using internet_search or extract_text_from_url):\n"
    "1. Every factual statement obtained from these two tools must include a "
    "citation number like this: [1], [2], etc.\n"
    "2. At the END of your response, include a 'Sources:' section.\n"
    "3. In that section, list each cited source using this exact format:\n"
    "      [1] <URL>\n"
    "      [2] <URL>\n"
    "4. Each numbered citation must correspond to a URL from the tool results.\n"
    "5. If the response does NOT require or use these two tools, do NOT include any citations.\n\n"
    
    "These rules apply *only* to the tools internet_search and extract_text_from_url. "
    "If other tools are added later, they are not affected by these citation rules unless specified.\n\n"
    
    "Be accurate, helpful, logical, always answer in markdown and follow citation rules strictly when required."
)


In [ ]:
def call_llm_node(state: MessagesState):
    
    messages = state["messages"]
    if not any(msg.__class__.__name__ == "SystemMessage" for msg in messages):
        messages.insert(0, system_msg)
        
    print("\n" + "-----"*10 + " All Messages content " + "-----"*10)
    for msg in messages:
        if isinstance(msg, SystemMessage):
            print("system msg =", msg.content)
        elif isinstance(msg, HumanMessage):
            print("human msg =", msg.content)
        # elif isinstance(msg, ToolMessage):
        #     print(f"tool msg = {msg.content}")
        elif isinstance(msg, AIMessage):
            print("assistant msg =", msg.content)
            print("usage_metadata = ", msg.usage_metadata)
    print("-----"*10 + "End of All Messages content " + "-----"*10 + "\n")

    # print("-----"*50)
    # print(f"\nAll Messages = {"messages"}\n")
    print("-----"*10 + " Last Message content " + "-----"*10)
    print(f"\nLast message = {messages[-1]}\n")
    print("-----"*10 + "End of Last Message content " + "-----"*10)
    response = llmwithtools.invoke(messages)
    return {"messages": [response]}

In [ ]:
def execute_tool_calls_node(state: MessagesState):
    last_message = state["messages"][-1]
    tool_outputs = []
    print("\n" +"-----"*10 + " Tool Calls to be executed " + "-----"*10)
    for tool_call in last_message.tool_calls:
        
        if tool_call["name"] == "internet_search":
            print("Invoking internet_search with args:", tool_call["args"])
            result = internet_search.invoke(tool_call["args"])
            tool_outputs.append(ToolMessage(tool_call_id=tool_call['id'], content=str(result)))
            
        elif tool_call["name"] == "extract_text_from_url":
            print("Invoking extract_text_from_url with args:", tool_call["args"])
            result = extract_text_from_url.invoke(tool_call["args"])
            tool_outputs.append(ToolMessage(tool_call_id=tool_call['id'], content=str(result)))
    
    print("-----"*10 + " End of Tool Calls execution " + "-----"*10 + "\n")
            
    return {"messages": tool_outputs}


In [ ]:
def should_call_tools(state: MessagesState):
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "tools"
    else:
        return "end"

# GRAPH

In [ ]:
graph = StateGraph(MessagesState)
graph.add_node("llm_node", call_llm_node)
graph.add_node("execute_tool_calls_node", execute_tool_calls_node)

graph.add_edge(START,"llm_node")
graph.add_conditional_edges(
    "llm_node", 
    should_call_tools,
    {
        "tools": "execute_tool_calls_node",
        "end": END,
    }
)
graph.add_edge("execute_tool_calls_node", "llm_node")


checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "ARJ"}}

In [ ]:
# use the compiled app (CompiledStateGraph) which exposes the graph
display(Image(app.get_graph().draw_mermaid_png()))

# RESULTS

In [ ]:
input1 = {"messages": [HumanMessage(content="before bye writea python code of for loop of start pattern")]}

In [ ]:
async for event in app.astream_events(input1, config):
    if event["event"] == "on_chat_model_stream":
        chunk = event["data"]["chunk"].content
        if chunk:
            print(chunk, end="", flush=True)

# CHECKPOINT TESTING 
### To view last AI message

In [ ]:
latest_checkpoint = checkpointer.get(config)
messages = latest_checkpoint["channel_values"]["messages"]
# print(messages)
last_ai_msg = None
for msg in reversed(messages):
    if msg.__class__.__name__ == "AIMessage":
        last_ai_msg = msg.content
        break
print("*"*100)
print(last_ai_msg)
print(messages[0].__class__.__name__)

In [ ]:
display_markdown(Markdown(last_ai_msg))